## How to run more than one function at the same time
Long operations on multiple instruments at the same time are sometimes unavoidable. Often these involve the local CPU idling, waiting for I/O operations to complete on the remote instrument. In these cases, it is convenient to be able to use threading to execute more than one of these operations at the same time. This is one type of concurrency. 

`labbench` includes simple concurrency support for this kind of I/O-constrained operation. It is not suitable for parallelizing CPU-intensive tasks because it relies on threading (which shares a single process on one CPU core) instead of multiprocessing (which may be able to spread operations across multiple CPU cores).

### Example
Here are very fake functions that just use `time.sleep` to block. They simulate longer instrument calls (such as triggering or acquisition) that take some time to complete.

Notice that `do_something_3` takes 3 arguments (and returns them), and that `do_something_4` raises an exception.

In [1]:
import time

def do_something_1 ():
    print 'start 1'
    time.sleep(1)
    print 'end 1'
    return 1

def do_something_2 ():
    print 'start 2'
    time.sleep(2)
    print 'end 2'
    return 2

def do_something_3 (a,b,c):
    print 'start 3'
    time.sleep(2.5)
    print 'end 3'
    return a,b,c 

def do_something_4 ():
    print 'start 4'
    time.sleep(3)
    raise ValueError('I had an error')
    print 'end 4'
    return 4

def do_something_5 ():
    print 'start 5'
    time.sleep(4)
    raise IndexError('I had a different error')
    print 'end 5'
    return 4

Here is the simplest example, where we call functions `do_something_1` and `do_something_2` that take no arguments and raise no exceptions:

In [2]:
from labbench import concurrently

results = concurrently(do_something_1, do_something_2)

print 'results: ', results[do_something_1], results[do_something_2]

start 1
start 2
end 1
end 2
results:  1 2


In [3]:
do_something_1.__name__

'do_something_1'

We can also pass functions by wrapping the functions in `Call()`, which is a class designed for this purpose:

In [3]:
from labbench import concurrently, Call

results = concurrently(do_something_1, Call(do_something_3, 1,2,c=3))

print 'results: ', results[do_something_1], results[do_something_3]

start 1start 3

end 1
end 3results: 
 1 (1, 2, 3)


More than one of the functions running concurrently may raise exceptions. Tracebacks print to the screen, and by default `ConcurrentException` is also raised:

In [4]:
from labbench import concurrently, Call

results = concurrently(do_something_4, do_something_5)

print 'results: ', results[do_something_1], results[do_something_3]

start 4
start 5



Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\labbench\concurrency.py", line 40, in __call__
    self.result = self.func(*self.args, **self.kws)
  File "<ipython-input-1-d8f843dd1423>", line 24, in do_something_4
    raise ValueError('I had an error')
ValueError: I had an error


Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\labbench\concurrency.py", line 40, in __call__
    self.result = self.func(*self.args, **self.kws)
  File "<ipython-input-1-d8f843dd1423>", line 31, in do_something_5
    raise IndexError('I had a different error')
IndexError: I had a different error



ConcurrentException: 2 call raised exceptions

the `catch` flag changes concurrent exception handling behavior to return values of functions that did not raise exceptions (instead of raising `ConcurrentException`). The return dictionary only includes keys for functions that did not raise exceptions.

In [5]:
from labbench import concurrently, Call

results = concurrently(do_something_4, do_something_1, catch=True)

print 'results: ', results

start 4
start 1
end 1
results:  {<function do_something_1 at 0x00000000046B4048>: 1}



Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\labbench\concurrency.py", line 40, in __call__
    self.result = self.func(*self.args, **self.kws)
  File "<ipython-input-1-d8f843dd1423>", line 24, in do_something_4
    raise ValueError('I had an error')
ValueError: I had an error

